In [29]:
import os
import cv2
import shutil
import random
import subprocess as sp
import supervision as sv

from tqdm import tqdm
from ultralytics import YOLO
from SoccerNet.Downloader import SoccerNetDownloader

In [4]:
# mySoccerNetDownloader = SoccerNetDownloader(LocalDirectory="datasets/")
# mySoccerNetDownloader.password = "s0cc3rn3t"

# mySoccerNetDownloader.downloadDataTask(task="tracking", split=["train", "test"])

In [2]:
train_path = '/home/adityapg/work/Sports-Analysis/datasets/train'
test_path = '/home/adityapg/work/Sports-Analysis/datasets/test'
images_path = '/home/adityapg/work/Sports-Analysis/datasets/images'
labels_path = '/home/adityapg/work/Sports-Analysis/datasets/labels'

config_path = '/home/adityapg/work/Sports-Analysis/Player Tracking/config.yaml'
model_path = '/home/adityapg/work/Sports-Analysis/models/yolov10m.pt'
finetune_path = '/home/adityapg/work/Sports-Analysis/Player Tracking/runs/detect/train/weights/best.pt'

In [4]:
def convert(box, width, height):
    x, y, w, h = box
    x, w = x / width, w / width
    y, h = y / height, h / height

    bbox = [x + w/2, y + h/2, w, h]
    return bbox

In [5]:
def make_labels(train_path):
    for name in os.listdir(train_path):
        for file in os.listdir(f'{train_path}/{name}/img1'):
            os.rename(f'{train_path}/{name}/img1/{file}', f'{train_path}/{name}/img1/{name}_{file}')

    for name in os.listdir(train_path):
        label_path = f'{train_path}/{name}/labels'    
        
        if os.path.exists(label_path):
            shutil.rmtree(label_path)
        os.makedirs(label_path)

        width, height = 1, 1
        with open(f'{train_path}/{name}/seqinfo.ini', 'r') as file:
            data = file.readlines()
            for line in data:
                if 'imWidth' in line:
                    width = eval(line.split('=')[1])
                elif 'imHeight' in line:
                    height = eval(line.split('=')[1])

        ball_ids = []
        goalkeeper_ids = []
        referee_ids = []
        with open(f'{train_path}/{name}/gameinfo.ini', 'r') as file:
            data = file.readlines()
            for line in data:
                if 'ball' in line:
                    ball_ids.append(eval(line.split('=')[0].split('_')[1]))
                if 'goalkeeper' in line:
                    goalkeeper_ids.append(eval(line.split('=')[0].split('_')[1]))
                if 'referee' in line:
                    referee_ids.append(eval(line.split('=')[0].split('_')[1]))

        with open(f'{train_path}/{name}/gt/gt.txt', 'r') as file:
            data = file.readlines()
            for line in data:
                line = [eval(i) for i in line.split(',')]
                bbox = convert(line[2:6], width, height)
                
                class_id = 1
                if line[1] in ball_ids:
                    class_id = 0
                elif line[1] in goalkeeper_ids:
                    class_id = 2
                elif line[1] in referee_ids:
                    class_id = 3

                with open(f'{train_path}/{name}/labels/{name}_{line[0]:06d}.txt', 'a') as f:
                    f.write(f'{class_id} {" ".join([str(i) for i in bbox])}\n')

In [6]:
def split(train_path, images_path, labels_path):
    if os.path.exists(images_path):
        shutil.rmtree(images_path)
    os.makedirs(images_path)

    os.makedirs(f'{images_path}/train')
    os.makedirs(f'{images_path}/val')

    if os.path.exists(labels_path):
        shutil.rmtree(labels_path)
    os.makedirs(labels_path)

    os.makedirs(f'{labels_path}/train')
    os.makedirs(f'{labels_path}/val')

    names = []
    for name in os.listdir(train_path):
        names.extend(os.listdir(f'{train_path}/{name}/img1'))

    names = [name.split('.')[0] for name in names]
    random.shuffle(names)
    train = names[:int(0.8*len(names))]

    for name in os.listdir(train_path):
        img_path = f'{train_path}/{name}/img1'
        label_path = f'{train_path}/{name}/labels'

        for file in os.listdir(img_path):
            file = file.split('.')[0]

            if file in train:
                shutil.copy(f'{img_path}/{file}.jpg', f'{images_path}/train/')
                shutil.copy(f'{label_path}/{file}.txt', f'{labels_path}/train/')
            else:
                shutil.copy(f'{img_path}/{file}.jpg', f'{images_path}/val/')
                shutil.copy(f'{label_path}/{file}.txt', f'{labels_path}/val/')

In [7]:
make_labels(train_path)
split(train_path, images_path, labels_path)

In [ ]:
model = YOLO(model_path)
model.train(data=config_path, epochs=5, batch=4, amp=False)

In [40]:
def make_video(clip_path):
    ffmpeg_path = 'ffmpeg'
    command = [
        ffmpeg_path,
        '-i', f'{clip_path}/img1/%06d.jpg',
        '-r', '25',
        '-y',
        '-v', 'quiet',
        '-c:v', 'libx264',
        f'{clip_path}/out.mp4'
    ]
    sp.run(command)

def annotate_video(clip_path, confidence=0.3):
    SOURCE_VIDEO_PATH = f'{clip_path}/out.mp4'
    TARGET_VIDEO_PATH = f'{clip_path}/annotated.mp4'

    box_annotator = sv.BoxAnnotator(
        color=sv.ColorPalette.from_hex(['#FF8C00', '#00BFFF', '#FF1493', '#FFD700']),
        thickness=2
    )
    label_annotator = sv.LabelAnnotator(
        color=sv.ColorPalette.from_hex(['#FF8C00', '#00BFFF', '#FF1493', '#FFD700']),
        text_color=sv.Color.from_hex('#000000')
    )

    video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
    video_sink = sv.VideoSink(TARGET_VIDEO_PATH, video_info=video_info)

    frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

    model = YOLO(finetune_path)
    with video_sink:
        for frame in tqdm(frame_generator, total=video_info.total_frames):
            result = model(frame, conf=confidence, verbose=False)[0]
            detections = sv.Detections.from_ultralytics(result)
            annotated_frame = frame.copy()
            annotated_frame = box_annotator.annotate(annotated_frame, detections)
            annotated_frame = label_annotator.annotate(annotated_frame, detections)

            video_sink.write_frame(annotated_frame)

In [41]:
clip_path = f'{test_path}/SNMOT-118'

make_video(clip_path)
annotate_video(clip_path)

100%|██████████| 750/750 [00:20<00:00, 37.28it/s]
